In [1]:
import os
import re
import pandas as pd

# === 1. Set Base Case Folder ===
base_path = "/home/jovyan/my_android_logs/CASE_FILES_raw_logs"

def is_valid_hostname(domain):
    return re.match(r"^[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", domain) is not None

# === 2. List available case folders ===
cases = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
print("📁 Available Case Folders:")
for folder in cases:
    print("→", folder)

# === 3. Ask user to select a case folder ===
selected_case = input("\nEnter exact case folder name: ").strip()
case_path = os.path.join(base_path, selected_case)

if not os.path.exists(case_path):
    raise FileNotFoundError(f"❌ Case folder '{selected_case}' does not exist!")

# === 4. List .txt and .xlsx log files in the folder ===
log_files = [f for f in os.listdir(case_path) if f.endswith(".txt") or f.endswith(".xlsx")]
print("\n📄 Available Log Files (.txt or .xlsx) in Case:")
for f in log_files:
    print("→", f)

# === 5. Ask user to select the log file ===
selected_log = input("\nEnter exact log file name to parse: ").strip()
full_log_path = os.path.join(case_path, selected_log)

if not os.path.isfile(full_log_path):
    raise FileNotFoundError(f"❌ File '{selected_log}' not found in case folder '{selected_case}'.")

parsed_logs = []

# === 6A. Parse `.txt` log files (logcat format) ===
if selected_log.endswith(".txt"):
    with open(full_log_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    log_pattern = re.compile(
    r"(?P<timestamp>\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d+)\s+"     # timestamp
    r"(?P<level>[A-Z])/"                                        # log level (D/I/W/E)
    r"(?P<tag>[^\(]+)"                                          # tag before PID
    r"\(\s*(?P<pid>\d+)\):\s+"                                  # PID
    r"(?P<message>.*)"                                          # message
)


    for line in lines:
        match = log_pattern.match(line)
        if match:
            data = match.groupdict()
            msg = data["message"]

            # More flexible class/method extraction
            class_method = re.findall(r'\b[\w$]+\.[\w$]+\([^)]*\)', msg)
            data["class_method"] = class_method[0] if class_method else None

            # More flexible domain extraction
            domain_match = re.search(r"((?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,})", msg)
            domain = domain_match.group(1) if domain_match else None
            data["domain"] = domain if domain and is_valid_hostname(domain) else None

            # Add missing fields for consistency with Excel
            data["record_class"] = None
            data["ip"] = None

            parsed_logs.append(data)

# === 6B. Parse `.xlsx` Excel DNS log format ===
elif selected_log.endswith(".xlsx"):
    df_excel = pd.read_excel(full_log_path)

    required_cols = ["Timestamp", "Query Domain"]
    for col in required_cols:
        if col not in df_excel.columns:
            raise ValueError(f"❌ Required column '{col}' not found in Excel file.")

    for _, row in df_excel.iterrows():
        data = {
            "timestamp": row.get("Timestamp", ""),
            "domain": row.get("Query Domain", ""),
            "record_type": row.get("Record Type", ""),
            "record_class": row.get("Record Class", ""),
            "server": row.get("Server", ""),
            "service": row.get("Service", ""),
            "client_ip": row.get("Client IP", ""),
            "port": row.get("Port", ""),
            "class_method": None,
            "pid": None,
            "ip": None
        }

        if not is_valid_hostname(str(data["domain"])):
            data["domain"] = None

        parsed_logs.append(data)

else:
    raise ValueError("❌ Unsupported file type! Only .txt and .xlsx are supported.")

# === 7. Load parsed data into DataFrame ===
df = pd.DataFrame(parsed_logs)
print(f"\n✅ Parsed {len(df)} log entries.\n")
print(df[["timestamp", "pid", "domain", "record_class", "class_method", "ip", "message"] if "message" in df.columns else df.columns.tolist()])


📁 Available Case Folders:
→ case_03
→ case_05
→ case_456_2025-06-27_17-48-19
→ trainning_case



Enter exact case folder name:  case_456_2025-06-27_17-48-19



📄 Available Log Files (.txt or .xlsx) in Case:
→ app_activity_logs.txt
→ dns_logs.txt
→ full_app_logs.txt



Enter exact log file name to parse:  app_activity_logs.txt



✅ Parsed 1392 log entries.

               timestamp    pid                      domain record_class  \
0     06-13 19:30:41.166   1178                        None         None   
1     06-15 14:21:17.993  30775               com.igdtuw.lc         None   
2     06-15 14:21:17.993  30775  java.lang.RuntimeException         None   
3     06-15 14:36:25.287   4704               com.igdtuw.lc         None   
4     06-15 14:36:25.287   4704  java.lang.RuntimeException         None   
...                  ...    ...                         ...          ...   
1387  06-27 17:48:14.433   6510                      IO.cpp         None   
1388  06-27 17:48:14.434   6510                        None         None   
1389  06-27 17:48:14.435   6510                      IO.cpp         None   
1390  06-27 17:48:14.449   6510                        None         None   
1391  06-27 17:48:14.451   6510                      IO.cpp         None   

     class_method    ip                                   

In [12]:
import socket

def resolve_domain(domain):                                       # create file for resolved dns-ip log
    try:
        ip = socket.gethostbyname(domain)
        return ip
    except Exception as e:
        return None

# Apply domain → IP resolution
df["ip"] = df["domain"].apply(lambda d: resolve_domain(d) if pd.notna(d) else None)

# Check if any IPs were resolved
if df["ip"].notna().sum() == 0:
    print("⚠️ No IPs were resolved from the domains.")
else:
    print(f"✅ {df['ip'].notna().sum()} IPs resolved from domains.")

# Ensure required columns exist
for col in ["timestamp", "domain", "record_class", "ip", "pid"]:
    if col not in df.columns:
        df[col] = None

# Prepare output
output_df = df[["timestamp", "domain", "record_class", "ip", "pid"]]

# Sort so that resolved IPs are on top
output_df = output_df.sort_values(by="ip", ascending=False, na_position='last')

# Save to CSV
output_file_path = os.path.join(case_path, "resolved_dns_log.csv")
output_df.to_csv(output_file_path, index=False)
print(f"📁 Resolved log saved to: {output_file_path}")


✅ 5 IPs resolved from domains.
📁 Resolved log saved to: /home/jovyan/my_android_logs/CASE_FILES_raw_logs/case_456_2025-06-27_17-48-19/resolved_dns_log.csv
